# Sales Dashboard Analysis

**Dataset**: 10,000 sales transactions (2023-2024)

---

## Project Overview

This project analyzes retail sales data to uncover business insights and create visualizations for a sales dashboard. The analysis covers:

1. **Revenue Trends** - Daily, monthly, and yearly patterns
2. **Product Performance** - Top products and categories
3. **Regional Analysis** - Sales by geographic region
4. **Customer Insights** - Customer segments and behavior
5. **Seasonal Patterns** - Holiday and seasonal trends

## 1. Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print('✅ Libraries loaded successfully!')

In [ ]:
# Load data
df = pd.read_csv('../data/sales_data.csv')

# Convert date column
df['date'] = pd.to_datetime(df['date'])

# Basic info
print(f"📊 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"📅 Date Range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"💰 Total Revenue: ${df['final_amount'].sum():,.2f}")
print(f"📦 Total Transactions: {len(df):,}")

In [ ]:
# Preview data
df.head(10)

In [ ]:
# Data types and info
df.info()

In [ ]:
# Statistical summary
df.describe()

## 2. Revenue Analysis

### 2.1 Monthly Revenue Trend

In [ ]:
# Monthly revenue
monthly_revenue = df.groupby(df['date'].dt.to_period('M'))['final_amount'].sum()
monthly_revenue.index = monthly_revenue.index.to_timestamp()

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(monthly_revenue.index, monthly_revenue.values, marker='o', linewidth=2, markersize=6)
ax.fill_between(monthly_revenue.index, monthly_revenue.values, alpha=0.3)

ax.set_title('📈 Monthly Revenue Trend (2023-2024)', fontsize=16, fontweight='bold')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Revenue ($)', fontsize=12)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Add average line
avg_revenue = monthly_revenue.mean()
ax.axhline(y=avg_revenue, color='red', linestyle='--', label=f'Average: ${avg_revenue:,.0f}')
ax.legend()

plt.tight_layout()
plt.savefig('../outputs/monthly_revenue_trend.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Average Monthly Revenue: ${avg_revenue:,.2f}")
print(f"📈 Highest Month: {monthly_revenue.idxmax().strftime('%B %Y')} - ${monthly_revenue.max():,.2f}")
print(f"📉 Lowest Month: {monthly_revenue.idxmin().strftime('%B %Y')} - ${monthly_revenue.min():,.2f}")

### 2.2 Year-over-Year Comparison

In [ ]:
# Yearly comparison
yearly_revenue = df.groupby('year')['final_amount'].agg(['sum', 'count', 'mean'])
yearly_revenue.columns = ['Total Revenue', 'Transactions', 'Avg Transaction']

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Revenue by year
colors = ['#3498db', '#e74c3c']
bars = axes[0].bar(yearly_revenue.index.astype(str), yearly_revenue['Total Revenue'], color=colors, edgecolor='black')
axes[0].set_title('💰 Total Revenue by Year', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Revenue ($)')
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Add values on bars
for bar, val in zip(bars, yearly_revenue['Total Revenue']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50000, 
                 f'${val/1e6:.2f}M', ha='center', fontsize=12, fontweight='bold')

# Transactions by year
bars2 = axes[1].bar(yearly_revenue.index.astype(str), yearly_revenue['Transactions'], color=colors, edgecolor='black')
axes[1].set_title('📦 Total Transactions by Year', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Number of Transactions')

for bar, val in zip(bars2, yearly_revenue['Transactions']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                 f'{val:,}', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/yearly_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# YoY Growth
if len(yearly_revenue) > 1:
    yoy_growth = ((yearly_revenue['Total Revenue'].iloc[1] - yearly_revenue['Total Revenue'].iloc[0]) 
                  / yearly_revenue['Total Revenue'].iloc[0] * 100)
    print(f"\n📈 Year-over-Year Revenue Growth: {yoy_growth:+.1f}%")

### 2.3 Daily Revenue Pattern (Day of Week)

In [ ]:
# Revenue by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_revenue = df.groupby('day_of_week')['final_amount'].agg(['sum', 'mean', 'count'])
daily_revenue = daily_revenue.reindex(day_order)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
colors = plt.cm.Blues(np.linspace(0.4, 0.9, 7))
bars = ax.bar(daily_revenue.index, daily_revenue['sum'], color=colors, edgecolor='black')

ax.set_title('📅 Revenue by Day of Week', fontsize=16, fontweight='bold')
ax.set_xlabel('Day of Week', fontsize=12)
ax.set_ylabel('Total Revenue ($)', fontsize=12)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Highlight best day
best_day_idx = daily_revenue['sum'].argmax()
bars[best_day_idx].set_color('#27ae60')
bars[best_day_idx].set_edgecolor('black')

plt.tight_layout()
plt.savefig('../outputs/daily_revenue.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n🏆 Best Sales Day: {daily_revenue['sum'].idxmax()} (${daily_revenue['sum'].max():,.2f})")
print(f"📉 Slowest Day: {daily_revenue['sum'].idxmin()} (${daily_revenue['sum'].min():,.2f})")

## 3. Product & Category Analysis

### 3.1 Revenue by Category

In [ ]:
# Category analysis
category_stats = df.groupby('category').agg({
    'final_amount': ['sum', 'mean', 'count'],
    'quantity': 'sum'
}).round(2)
category_stats.columns = ['Total Revenue', 'Avg Transaction', 'Transactions', 'Units Sold']
category_stats = category_stats.sort_values('Total Revenue', ascending=False)
category_stats['Revenue %'] = (category_stats['Total Revenue'] / category_stats['Total Revenue'].sum() * 100).round(1)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
colors = ['#e74c3c', '#3498db', '#27ae60', '#f39c12', '#9b59b6']
explode = (0.05, 0, 0, 0, 0)
axes[0].pie(category_stats['Total Revenue'], labels=category_stats.index, autopct='%1.1f%%',
            colors=colors, explode=explode, shadow=True, startangle=90)
axes[0].set_title('🥧 Revenue Share by Category', fontsize=14, fontweight='bold')

# Bar chart
bars = axes[1].barh(category_stats.index, category_stats['Total Revenue'], color=colors, edgecolor='black')
axes[1].set_title('📊 Revenue by Category', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Revenue ($)')
axes[1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
axes[1].invert_yaxis()

# Add values
for bar, val in zip(bars, category_stats['Total Revenue']):
    axes[1].text(val + 30000, bar.get_y() + bar.get_height()/2, 
                 f'${val/1e6:.2f}M', va='center', fontsize=11)

plt.tight_layout()
plt.savefig('../outputs/category_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📈 Category Performance Summary:")
display(category_stats)

### 3.2 Top 10 Products by Revenue

In [ ]:
# Top products
top_products = df.groupby('product_name').agg({
    'final_amount': 'sum',
    'quantity': 'sum',
    'transaction_id': 'count'
}).rename(columns={'transaction_id': 'orders'})
top_products = top_products.sort_values('final_amount', ascending=False).head(10)

# Plot
fig, ax = plt.subplots(figsize=(12, 7))
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 10))[::-1]
bars = ax.barh(top_products.index, top_products['final_amount'], color=colors, edgecolor='black')

ax.set_title('🏆 Top 10 Products by Revenue', fontsize=16, fontweight='bold')
ax.set_xlabel('Revenue ($)', fontsize=12)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
ax.invert_yaxis()

# Add values
for bar, val in zip(bars, top_products['final_amount']):
    ax.text(val + 10000, bar.get_y() + bar.get_height()/2, 
            f'${val:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/top_products.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🏆 Top 10 Products:")
display(top_products)

## 4. Regional Analysis

In [ ]:
# Regional analysis
regional_stats = df.groupby('region').agg({
    'final_amount': ['sum', 'mean', 'count'],
    'quantity': 'sum'
})
regional_stats.columns = ['Total Revenue', 'Avg Transaction', 'Transactions', 'Units Sold']
regional_stats = regional_stats.sort_values('Total Revenue', ascending=False)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#e74c3c', '#3498db', '#27ae60', '#f39c12']

# Revenue by region
bars = axes[0].bar(regional_stats.index, regional_stats['Total Revenue'], color=colors, edgecolor='black')
axes[0].set_title('🗺️ Revenue by Region', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Revenue ($)')
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

for bar, val in zip(bars, regional_stats['Total Revenue']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20000, 
                 f'${val/1e6:.2f}M', ha='center', fontsize=11, fontweight='bold')

# Transactions by region
bars2 = axes[1].bar(regional_stats.index, regional_stats['Transactions'], color=colors, edgecolor='black')
axes[1].set_title('📦 Transactions by Region', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Number of Transactions')

for bar, val in zip(bars2, regional_stats['Transactions']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30, 
                 f'{val:,}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/regional_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n🗺️ Regional Performance:")
display(regional_stats)

### 4.1 Regional Category Heatmap

In [ ]:
# Cross-tabulation: Region vs Category
pivot_data = df.pivot_table(values='final_amount', index='region', columns='category', aggfunc='sum')

# Heatmap
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(pivot_data, annot=True, fmt=',.0f', cmap='YlOrRd', 
            linewidths=0.5, ax=ax, cbar_kws={'label': 'Revenue ($)'})

ax.set_title('🔥 Revenue Heatmap: Region × Category', fontsize=16, fontweight='bold')
ax.set_xlabel('Category', fontsize=12)
ax.set_ylabel('Region', fontsize=12)

plt.tight_layout()
plt.savefig('../outputs/region_category_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Customer Analysis

In [ ]:
# Customer type analysis
customer_stats = df.groupby('customer_type').agg({
    'final_amount': ['sum', 'mean', 'count']
})
customer_stats.columns = ['Total Revenue', 'Avg Transaction', 'Transactions']
customer_stats = customer_stats.sort_values('Total Revenue', ascending=False)
customer_stats['Revenue %'] = (customer_stats['Total Revenue'] / customer_stats['Total Revenue'].sum() * 100).round(1)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#27ae60', '#3498db', '#e74c3c']

# Pie chart
axes[0].pie(customer_stats['Total Revenue'], labels=customer_stats.index, autopct='%1.1f%%',
            colors=colors, explode=[0.02]*3, shadow=True, startangle=90)
axes[0].set_title('👥 Revenue by Customer Type', fontsize=14, fontweight='bold')

# Bar chart - avg transaction
bars = axes[1].bar(customer_stats.index, customer_stats['Avg Transaction'], color=colors, edgecolor='black')
axes[1].set_title('💵 Average Transaction by Customer Type', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Average Transaction ($)')

for bar, val in zip(bars, customer_stats['Avg Transaction']):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                 f'${val:.2f}', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/customer_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n👥 Customer Segment Performance:")
display(customer_stats)

## 6. Payment Method Analysis

In [ ]:
# Payment method analysis
payment_stats = df.groupby('payment_method').agg({
    'final_amount': ['sum', 'count']
})
payment_stats.columns = ['Total Revenue', 'Transactions']
payment_stats = payment_stats.sort_values('Transactions', ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.Set3(np.linspace(0, 1, len(payment_stats)))

ax.pie(payment_stats['Transactions'], labels=payment_stats.index, autopct='%1.1f%%',
       colors=colors, explode=[0.02]*len(payment_stats), shadow=True, startangle=90)
ax.set_title('💳 Transactions by Payment Method', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/payment_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💳 Payment Method Usage:")
display(payment_stats)

## 7. Discount Analysis

In [ ]:
# Discount analysis
df['has_discount'] = df['discount_percent'] > 0

discount_summary = df.groupby('has_discount').agg({
    'final_amount': ['sum', 'mean', 'count'],
    'discount_amount': 'sum'
})
discount_summary.columns = ['Total Revenue', 'Avg Transaction', 'Transactions', 'Total Discount']
discount_summary.index = ['No Discount', 'With Discount']

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Discount vs No Discount
colors = ['#3498db', '#e74c3c']
bars = axes[0].bar(discount_summary.index, discount_summary['Transactions'], color=colors, edgecolor='black')
axes[0].set_title('🏷️ Transactions: Discount vs No Discount', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Transactions')

for bar, val in zip(bars, discount_summary['Transactions']):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                 f'{val:,}', ha='center', fontsize=12, fontweight='bold')

# Discount distribution
discounted_df = df[df['discount_percent'] > 0]
axes[1].hist(discounted_df['discount_percent'], bins=5, color='#e74c3c', edgecolor='black', alpha=0.7)
axes[1].set_title('📊 Discount Percentage Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Discount %')
axes[1].set_ylabel('Number of Transactions')

plt.tight_layout()
plt.savefig('../outputs/discount_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

total_discount = df['discount_amount'].sum()
print(f"\n🏷️ Total Discount Given: ${total_discount:,.2f}")
print(f"📊 Transactions with Discount: {len(discounted_df):,} ({len(discounted_df)/len(df)*100:.1f}%)")

## 8. Key Insights & Recommendations

In [ ]:
# Generate key metrics
print("="*60)
print("📊 SALES DASHBOARD - KEY INSIGHTS")
print("="*60)

print("\n💰 REVENUE METRICS")
print(f"   • Total Revenue: ${df['final_amount'].sum():,.2f}")
print(f"   • Average Transaction: ${df['final_amount'].mean():,.2f}")
print(f"   • Total Transactions: {len(df):,}")

print("\n🏆 TOP PERFORMERS")
print(f"   • Best Category: {category_stats.index[0]} (${category_stats['Total Revenue'].iloc[0]:,.2f})")
print(f"   • Best Region: {regional_stats.index[0]} (${regional_stats['Total Revenue'].iloc[0]:,.2f})")
print(f"   • Best Product: {top_products.index[0]} (${top_products['final_amount'].iloc[0]:,.2f})")

print("\n📈 RECOMMENDATIONS")
print("   1. Focus marketing on Electronics category (highest revenue)")
print(f"   2. Expand presence in {regional_stats.index[0]} region (top performer)")
print("   3. Target VIP customers with exclusive offers (highest avg transaction)")
print(f"   4. Optimize discount strategy ({len(discounted_df)/len(df)*100:.0f}% transactions use discounts)")
print("\n" + "="*60)

---

## 💾 Export Summary Data

In [ ]:
# Create summary DataFrame
summary_data = {
    'Metric': [
        'Total Revenue',
        'Total Transactions',
        'Average Transaction Value',
        'Total Units Sold',
        'Total Discount Given',
        'Top Category',
        'Top Region',
        'Top Product'
    ],
    'Value': [
        f"${df['final_amount'].sum():,.2f}",
        f"{len(df):,}",
        f"${df['final_amount'].mean():,.2f}",
        f"{df['quantity'].sum():,}",
        f"${df['discount_amount'].sum():,.2f}",
        category_stats.index[0],
        regional_stats.index[0],
        top_products.index[0]
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('../outputs/dashboard_summary.csv', index=False)
print("✅ Summary exported to dashboard_summary.csv")
display(summary_df)

---

**Project Complete!** 🎉

All visualizations have been saved to the `outputs/` folder.